In [3]:
import tensorflow as tf 
import cv2
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical


In [25]:
#read the training data 
import random 
random.shuffle(training_data)
train_dir = r'F:\it18021\archive\train'
classes=['0','1','2','3','4','5','6']
img_size=224
training_data=[]
for category in classes :
    path = os.path.join(train_dir,category)
    class_num = classes.index(category)
    for img in os.listdir(path):
        try: 
            img_array= cv2.imread(os.path.join(path,img))
            new_array= cv2.resize(img_array,(img_size,img_size))
            training_data.append([new_array,class_num])
        except Exception as e:
            print(e)
            pass
X=[] #feature 
y=[] #label emotion 
for features,label in training_data:
    X.append(features)
    y.append(label)
print(len(X))
print(len(y))
X=np.array(X).reshape(-1,img_size,img_size,3)
X=X/255.0 #rescaling 
y=np.array(y)
print(X.shape,y.shape)

In [52]:
test_dir = r'F:\it18021\archive\test'
img_size=224
test_data=[]
for category in classes :
    path = os.path.join(test_dir,category)
    class_num = classes.index(category)
    for img in os.listdir(path):
        try: 
            img_array= cv2.imread(os.path.join(path,img))
            new_array= cv2.resize(img_array,(img_size,img_size))
            test_data.append([new_array,class_num])
        except Exception as e:
            print(e)
            pass
print(len(test_data))
X_test=[]
y_test=[]
for features,label in test_data:
    X_test.append(features)
    y_test.append(label)
print(len(X_test))
print(len(y_test))
X_test = np.array(X_test).reshape(-1,img_size,img_size,3)
X_test = X_test/255.0
y_test=np.array(y_test)
print(X_test.shape)
print(y_test.shape)

7178
7178
7178
(7178, 224, 224, 3)
(7178,)


In [58]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.7): # Experiment with changing this value
          print("\nReached 70% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [59]:
#defining the model 
#transfer learning 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [60]:
model= tf.keras.applications.MobileNetV2() 

In [61]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [62]:
model.trainable=False # freezing the layer 
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [63]:
base_input= model.layers[0].input

In [64]:
base_output=model.layers[-2].output

In [65]:
base_input

<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>

In [66]:
base_output

<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(None, 1280) dtype=float32>

In [67]:
final_output= layers.Dense(128)(base_output)
final_output= layers.Activation('relu')(final_output)
final_output= layers.Dense(64)(final_output)
final_output= layers.Activation('relu')(final_output)
final_output= layers.Dense(7,activation='softmax')(final_output)

In [68]:
final_output

<tf.Tensor 'dense_5/Softmax:0' shape=(None, 7) dtype=float32>

In [69]:
new_model= keras.Model(inputs= base_input,outputs=final_output)

In [70]:
new_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [71]:
new_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [72]:
new_model.fit(X,y,epochs=30,callbacks=[callbacks])

Epoch 1/30
898/898 [==============================] - 300s 334ms/step - loss: 1.4820 - accuracy: 0.4327
Epoch 2/30
898/898 [==============================] - 277s 309ms/step - loss: 1.3393 - accuracy: 0.4919
Epoch 3/30
898/898 [==============================] - 278s 310ms/step - loss: 1.2789 - accuracy: 0.5145
Epoch 4/30
898/898 [==============================] - 269s 299ms/step - loss: 1.2296 - accuracy: 0.5354
Epoch 5/30
898/898 [==============================] - 260s 290ms/step - loss: 1.1989 - accuracy: 0.5464
Epoch 6/30
898/898 [==============================] - 257s 287ms/step - loss: 1.1559 - accuracy: 0.5647
Epoch 7/30
898/898 [==============================] - 258s 287ms/step - loss: 1.1211 - accuracy: 0.5771
Epoch 8/30
898/898 [==============================] - 258s 287ms/step - loss: 1.0882 - accuracy: 0.5925
Epoch 9/30
898/898 [==============================] - 258s 288ms/step - loss: 1.0415 - accuracy: 0.6112
Epoch 10/30
898/898 [==============================] - 258s 287m

In [75]:
new_model.save('my_model65.h5')

In [76]:
new_model.evaluate(X_test,y_test)

225/225 [==============================] - 65s 291ms/step - loss: 1.4856 - accuracy: 0.5242


[1.4856343269348145, 0.5242407321929932]